# EEEN30131 Power System Analysis: Week 01 - Nodal Analysis

***&copy; 2024 Martínez Ceseña — University of Manchester, UK***

This notebook provides an example of the shortcut approach used to build a $Y_{bus}$ matrix, including general `python` code which can be used to solve the examples and create new ones.

The use of the notebooks is optional and will not be marked. That said, you are strongly encouraged to play with the tools and examples, as you can explore many different variations of the power flow formulation, which will better prepare you for the exams.

## List of contents

- [Building admittance matrix](#Building-admittance-matrix)
  - [Defining the problem](#Defining-the-problem)
  - [Validating the data](#Validating-the-data)
  - [Using shortcut method](#Using-shortcut-method)
- [Developing python method](#Developing-python-method)
- [Examples](#Examples)
  - [Three bus example](#Three-bus-example)
  - [Four bus example](#Four-bus-example)
  - [Create your own example](#Create-your-own-example)

## Before we begin

Before we begin: 
- Make sure to review the asynchronous materials provided in blackboard for EEEN30131 Week 1 - Nodal analysis 
- If you have any questions, please post them in the discussion boards or, if that is not possible, send an email to alex.martinezcesena@manchester.ac.uk

## Building admittance matrix

### Defining the problem

Let us begin with the last example that was addressed in week 01, which is a 3bus poer system with three lines.

![Week01_3Bus_Part1.png](Figures/Week01_3Bus_Part1.png)

We need to define the data for the problem, which we will place in a matrix, called connectivity

| From | To | Impedance |
| --- | --- | --- |
| 1 | 2 | j0.10 |
| 1 | 3 | j0.20 |
| 2 | 3 | j0.25 |

This can be written in python as follows:

***Note that last row of the `Connectivity` matrix should not have a comma at the end***

In [1]:
Connectivity = [
    [1, 2, complex(0, 0.10)],
    [1, 3, complex(0, 0.20)],
    [2, 3, complex(0, 0.25)]
]

[Back to top](#EEEN30131-Power-System-Analysis:-Week-01---Nodal-Analysis)

### Validating the data

Now that we have the data coded in python, ww want to measure the information. We already know that the system has three lines and three buses, but we need to check this, as we may want to simulate other networks.

To get the number of branches, we can use the `len` method to get the length of the Connectivity matrix (i.e., the number of rows).

In [2]:
# Get number of branches
Number_Branches = len(Connectivity)

To get the number of buses, we will use a `for` loop to check every branch, and the `max` method to get the maximum bus number included in the Connectivity matrix.

In [3]:
# Get number of nodes
Number_Buses = 0
for branch in range(Number_Branches):
    Number_Buses = \
        max([Number_Buses, Connectivity[branch][0],
             Connectivity[branch][1]])

The code below is used to check that the data has been correctly added and measured.

In [4]:
print('The network has %d branches and %d buses'%(Number_Branches, Number_Buses))
print('______________________________')
print('Branch | From - To | Impedance')
print('------------------------------')
for branch in range(Number_Branches):
    print('%5.0f  | %4.0f - %2.0f |'%(branch+1, Connectivity[branch][0], Connectivity[branch][1]), end=' ')
    print(Connectivity[branch][2])
print('_______|___________|__________')

The network has 3 branches and 3 buses
______________________________
Branch | From - To | Impedance
------------------------------
    1  |    1 -  2 | 0.1j
    2  |    1 -  3 | 0.2j
    3  |    2 -  3 | 0.25j
_______|___________|__________


[Back to top](#EEEN30131-Power-System-Analysis:-Week-01---Nodal-Analysis)

### Using shortcut method

It is now time to develop the admittance ($Y_{bus}$) matrix. Le us begin by creating a 3x3 matrix (i.e., Number_Buses x Number_Buses).

We will use the `numpy` python library, which will allow us to define a matrix with complex numbers.

In [5]:
import numpy
Ybus = numpy.zeros((Number_Buses, Number_Buses), dtype=complex)

print('Ybus = \n', Ybus)

Ybus = 
 [[0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j]]


Based on the $Y_{bus}$ shortcut:
> Get diagonal elements as: the summation of admittances connected to the bus:

***Note that, if there is a reference bus (bus zero), is should be skipped***

In [6]:
for branch in range(Number_Branches):
    From = Connectivity[branch][0] - 1
    To = Connectivity[branch][1] - 1

    if From >= 0:
        Ybus[From][From] = Ybus[From][From] + 1/Connectivity[branch][2]

    if To >= 0:
        Ybus[To][To] = Ybus[To][To] + 1/Connectivity[branch][2]

print('Ybus = \n', Ybus)

Ybus = 
 [[0.-15.j 0. +0.j 0. +0.j]
 [0. +0.j 0.-14.j 0. +0.j]
 [0. +0.j 0. +0.j 0. -9.j]]


> Get off-diagonal elements as: minus the admittance of the branch

***Note that, if there are connections to a reference bus (bus zero), the connections would be skipped***

In [7]:
for branch in range(Number_Branches):
    From = Connectivity[branch][0] - 1
    To = Connectivity[branch][1] - 1

    if From >=0 and To >=0:
        Ybus[From][To] = -1/Connectivity[branch][2]
        Ybus[To][From] = -1/Connectivity[branch][2]

print('Ybus = \n', Ybus)

Ybus = 
 [[0.-15.j 0.+10.j 0. +5.j]
 [0.+10.j 0.-14.j 0. +4.j]
 [0. +5.j 0. +4.j 0. -9.j]]


[Back to top](#EEEN30131-Power-System-Analysis:-Week-01---Nodal-Analysis)

## Developing python method

For convenience, we can create a python method (let us call it `get_Ybus`) based on all the code that was presented above. This will allow us to solve different types of examples, as long as the data is provided in the right format, i.e., the format we created above for the `Connectivity` matrix.

In [8]:
def get_Ybus(Connectivity, flg=False, prnt=True):

    # Get number of branches
    Number_Branches = len(Connectivity)

    # Get number of nodes
    Number_Buses = 0
    for branch in range(Number_Branches):
        Number_Buses = \
            max([Number_Buses, Connectivity[branch][0],
                 Connectivity[branch][1]])

    # Display network data
    if prnt:
        print('The network has %d branches and %d buses' % (Number_Branches,
                                                            Number_Buses))
        print('______________________________')
        print('Branch | From - To | Impedance')
        print('------------------------------')
        for branch in range(Number_Branches):
            print('%5.0f  | %4.0f - %2.0f |' % (branch+1,
                                                Connectivity[branch][0],
                                                Connectivity[branch][1]),
                  end=' ')
            print(Connectivity[branch][2])
        print('_______|___________|__________')

    # Build Ybus matrix
    Ybus = numpy.zeros((Number_Buses, Number_Buses), dtype=complex)
    for branch in range(Number_Branches):
        From = Connectivity[branch][0] - 1
        To = Connectivity[branch][1] - 1

        Y = 1/Connectivity[branch][2]

        if From >= 0 and To >= 0:
            Ybus[From][To] = -Y
            Ybus[To][From] = -Y

        if From >= 0:
            Ybus[From][From] += Y

        if To >= 0:
            Ybus[To][To] += Y

    if prnt:
        print('\nYbus = \n', Ybus)
    if flg:
        return Ybus

This method can be called as follows:
> `get_Ybus(Connectivity)`

We can take this a step further and create a version of the method with detailed description of each step of the admittance matrix calculations.

In [9]:
def get_Ybus_Steps(Connectivity, flg=False):
    # Get number of branches
    Number_Branches = len(Connectivity)

    # Get number of nodes
    Number_Buses = 0
    for branch in range(Number_Branches):
        Number_Buses = \
            max([Number_Buses, Connectivity[branch][0],
                 Connectivity[branch][1]])

    # Display network data
    print('The network has %d branches and %d buses' % (Number_Branches,
                                                        Number_Buses))
    print('______________________________')
    print('Branch | From - To | Impedance')
    print('------------------------------')
    for branch in range(Number_Branches):
        print('%5.0f  | %4.0f - %2.0f |' % (branch+1, Connectivity[branch][0],
                                            Connectivity[branch][1]), end=' ')
        print(Connectivity[branch][2])
    print('_______|___________|__________')

    print('\nBUILDING ADMITTANCE MATRIX:')
    Ybus = numpy.zeros((Number_Buses, Number_Buses), dtype=complex)
    for branch in range(Number_Branches):
        F = Connectivity[branch][0]
        T = Connectivity[branch][1]
        From = F - 1
        To = T - 1

        Z = Connectivity[branch][2]
        Y = 1/Z

        print('\nChecking branch %d (%d-%d)' % (branch+1, F, T), end=' ')
        print('with an impedance of', Z, ' (admittance of', Y, ')')

        if From >= 0 and To >= 0:
            print('The value of the off-diagonal elements ', end='')
            print('(%d,%d) and (%d,%d) is:'
                  % (F, T, T, F), -Y)
            Ybus[From][To] = -Y
            Ybus[To][From] = -Y

        if From >= 0:
            print('The value of the diagonal element (%d,%d)' %
                  (F, F), ' changes from ', Ybus[From][From], end=' ')
            Ybus[From][From] += Y
            print('to', Ybus[From][From])

        if To >= 0:
            print('The value of the diagonal element (%d,%d)' %
                  (T, T), ' changes from ', Ybus[To][To], end=' ')
            Ybus[To][To] += Y
            print('to', Ybus[To][To])

    print('\nYbus = \n', Ybus)
    if flg:
        return Ybus

This new method can be called with the follwoing command:
> `get_Ybus_Steps(Connectivity)`

[Back to top](#EEEN30131-Power-System-Analysis:-Week-01---Nodal-Analysis)

## Examples

### Three bus example

Now that we have coded the method to build the admittance matrix, we can solve our 3bus example by defining the `Connectivity` matrix and calling the `get_Ybus` method.


<img src="Figures/Week01_3Bus_Part1.png" alt="Fig01" class="bg-primary" width="500px">

| From | To | Impedance |
| --- | --- | --- |
| 1 | 2 | j0.10 |
| 1 | 3 | j0.20 |
| 2 | 3 | j0.25 |

***Try changing some values and, perhaps, adding lines***

In [10]:
Connectivity = [
    [1, 2, complex(0, 0.10)],
    [1, 3, complex(0, 0.20)],
    [2, 3, complex(0, 0.25)]
]

get_Ybus(Connectivity)

The network has 3 branches and 3 buses
______________________________
Branch | From - To | Impedance
------------------------------
    1  |    1 -  2 | 0.1j
    2  |    1 -  3 | 0.2j
    3  |    2 -  3 | 0.25j
_______|___________|__________

Ybus = 
 [[ 0.-15.j -0.+10.j -0. +5.j]
 [-0.+10.j  0.-14.j -0. +4.j]
 [-0. +5.j -0. +4.j  0. -9.j]]


We can use the other method if we want more details about the calculations:

In [11]:
get_Ybus_Steps(Connectivity)

The network has 3 branches and 3 buses
______________________________
Branch | From - To | Impedance
------------------------------
    1  |    1 -  2 | 0.1j
    2  |    1 -  3 | 0.2j
    3  |    2 -  3 | 0.25j
_______|___________|__________

BUILDING ADMITTANCE MATRIX:

Checking branch 1 (1-2) with an impedance of 0.1j  (admittance of -10j )
The value of the off-diagonal elements (1,2) and (2,1) is: (-0+10j)
The value of the diagonal element (1,1)  changes from  0j to -10j
The value of the diagonal element (2,2)  changes from  0j to -10j

Checking branch 2 (1-3) with an impedance of 0.2j  (admittance of -5j )
The value of the off-diagonal elements (1,3) and (3,1) is: (-0+5j)
The value of the diagonal element (1,1)  changes from  -10j to -15j
The value of the diagonal element (3,3)  changes from  0j to -5j

Checking branch 3 (2-3) with an impedance of 0.25j  (admittance of -4j )
The value of the off-diagonal elements (2,3) and (3,2) is: (-0+4j)
The value of the diagonal element (2,2)  

[Back to top](#EEEN30131-Power-System-Analysis:-Week-01---Nodal-Analysis)

### Four bus example

Lets try another example presented in week 01.

![Week01_4Bus_Part1.png](Figures/Week01_4Bus_Part1.png)

| From | To | Impedance |
| --- | --- | --- |
| 1 | 2 | 0.125 |
| 1 | 3 | 0.250 |
| 1 | 4 | 0.400 |
| 2 | 3 | 0.250 |
| 2 | 4 | 0.200 |
| 3 | 0 | 1.250 |
| 4 | 0 | 1.250 |

***Try changing some values and, perhaps, adding lines***

In [12]:
Connectivity = [
    [1, 2, complex(0.125, 0)],
    [1, 3, complex(0.250, 0)],
    [1, 4, complex(0.400, 0)],
    [2, 3, complex(0.250, 0)],
    [2, 4, complex(0.200, 0)],
    [3, 0, complex(1.250, 0)],
    [4, 0, complex(1.250, 0)]
]
get_Ybus(Connectivity)

The network has 7 branches and 4 buses
______________________________
Branch | From - To | Impedance
------------------------------
    1  |    1 -  2 | (0.125+0j)
    2  |    1 -  3 | (0.25+0j)
    3  |    1 -  4 | (0.4+0j)
    4  |    2 -  3 | (0.25+0j)
    5  |    2 -  4 | (0.2+0j)
    6  |    3 -  0 | (1.25+0j)
    7  |    4 -  0 | (1.25+0j)
_______|___________|__________

Ybus = 
 [[14.5+0.j -8. -0.j -4. -0.j -2.5-0.j]
 [-8. -0.j 17. +0.j -4. -0.j -5. -0.j]
 [-4. -0.j -4. -0.j  8.8+0.j  0. +0.j]
 [-2.5-0.j -5. -0.j  0. +0.j  8.3+0.j]]


As above, we can use the other method to get mroe details about the calculations:

In [13]:
get_Ybus_Steps(Connectivity)

The network has 7 branches and 4 buses
______________________________
Branch | From - To | Impedance
------------------------------
    1  |    1 -  2 | (0.125+0j)
    2  |    1 -  3 | (0.25+0j)
    3  |    1 -  4 | (0.4+0j)
    4  |    2 -  3 | (0.25+0j)
    5  |    2 -  4 | (0.2+0j)
    6  |    3 -  0 | (1.25+0j)
    7  |    4 -  0 | (1.25+0j)
_______|___________|__________

BUILDING ADMITTANCE MATRIX:

Checking branch 1 (1-2) with an impedance of (0.125+0j)  (admittance of (8+0j) )
The value of the off-diagonal elements (1,2) and (2,1) is: (-8-0j)
The value of the diagonal element (1,1)  changes from  0j to (8+0j)
The value of the diagonal element (2,2)  changes from  0j to (8+0j)

Checking branch 2 (1-3) with an impedance of (0.25+0j)  (admittance of (4+0j) )
The value of the off-diagonal elements (1,3) and (3,1) is: (-4-0j)
The value of the diagonal element (1,1)  changes from  (8+0j) to (12+0j)
The value of the diagonal element (3,3)  changes from  0j to (4+0j)

Checking branch 3

[Back to top](#EEEN30131-Power-System-Analysis:-Week-01---Nodal-Analysis)

### Create your own example

Now that you have seen how the `get_Ybus` method works, you can create your own examples. ***Try it!***

In [14]:
Connectivity = [
    [1, 2, complex(0.1, 0.0)],
    [1, 3, complex(0.1, 0.0)],
    [1, 4, complex(0.1, 0.0)],
    [2, 3, complex(0.0, 0.2)],
    [2, 4, complex(0.0, 0.2)],
    [3, 4, complex(0.1, 0.2)]
]
get_Ybus_Steps(Connectivity)

The network has 6 branches and 4 buses
______________________________
Branch | From - To | Impedance
------------------------------
    1  |    1 -  2 | (0.1+0j)
    2  |    1 -  3 | (0.1+0j)
    3  |    1 -  4 | (0.1+0j)
    4  |    2 -  3 | 0.2j
    5  |    2 -  4 | 0.2j
    6  |    3 -  4 | (0.1+0.2j)
_______|___________|__________

BUILDING ADMITTANCE MATRIX:

Checking branch 1 (1-2) with an impedance of (0.1+0j)  (admittance of (10+0j) )
The value of the off-diagonal elements (1,2) and (2,1) is: (-10-0j)
The value of the diagonal element (1,1)  changes from  0j to (10+0j)
The value of the diagonal element (2,2)  changes from  0j to (10+0j)

Checking branch 2 (1-3) with an impedance of (0.1+0j)  (admittance of (10+0j) )
The value of the off-diagonal elements (1,3) and (3,1) is: (-10-0j)
The value of the diagonal element (1,1)  changes from  (10+0j) to (20+0j)
The value of the diagonal element (3,3)  changes from  0j to (10+0j)

Checking branch 3 (1-4) with an impedance of (0.1+0j) 

[Back to top](#EEEN30131-Power-System-Analysis:-Week-01---Nodal-Analysis)